# 💻 Statiske kart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GMGI221-2024/forelesninger/blob/main/11_statiske_kart.ipynb)

I løpet av de siste ukene har vi allerede blitt kjent med plotting av
grunnleggende statiske kart ved hjelp av
[`geopandas.GeoDataFrame.plot()`](http://geopandas.org/mapping.html), for
eksempel i leksjonene [4](#04_geopandas),
[7](#07_romlig_kobling), og [9](#09_reklassifisering). Vi lærte også
at `geopandas.GeoDataFrame.plot()` bruker `matplotlib.pyplot`
biblioteket, og at [de fleste av argumentene og alternativene blir akseptert av
geopandas](https://matplotlib.org/stable/api/pyplot_summary.html).

For å friske opp hukommelsen om det grunnleggende i å plotte kart, la oss lage et statisk
befolkningskart over Oslo, som også viser veier og
metro linjer (tre lag, overlappet hverandre). Husk at inputdataene må være i samme koordinatsystem!


## Data

Vi vil bruke tre forskjellige datasett:
- Befolkningsrutenettet for Oslo som vi brukte i [leksjon
  09](#09_reklassifisering), som ligger i `DATA_MAPPE /
"ssb_rutenett"`,
- T-bane nettverket i Oslo, som er hentet fra OpenStreetMap,
  og
- et forenklet nettverk av de største veiene i Oslo,
  også hentet fra OpenStreetMap.

In [19]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

In [20]:
import geopandas
import numpy

rutenett = geopandas.read_file(
    DATA_MAPPE
    / "ssb_rutenett"
    / ""
)

tbane = geopandas.read_file(DATA_MAPPE / "osm_data" / "")

veier = geopandas.read_file(DATA_MAPPE / "osm_data" / "")

:::{admonition} Koordinatreferansesystemer
:class: attention

Husk at forskjellige geo-dataframes må være i samme koordinatsystem
før de plottes på samme kart. `geopandas.GeoDataFrame.plot()` utfører ikke
reprosjisering av data automatisk.

Du kan alltid sjekke det med en enkel `assert` uttalelse.
:::

In [ ]:
assert rutenett.crs == tbane.crs == veier.crs, "Inndataenes CRS er forskjellige"

Hvis flere datasett ikke deler et felles CRS, finn ut først hvilket CRS
de har tilknyttet (hvis noe!), deretter transformere dataene til et felles referansesystem:

In [8]:
assert rutenett.crs == tbane.crs == veier.crs, "Inndataenes CRS er forskjellige"

## Plotting et flerlagskart

:::{admonition} Sjekk forståelsen din
:class: hint

Fullfør de neste trinnene i ditt eget tempo (tøm først kodecellene).
Sørg for å gå tilbake til tidligere leksjoner hvis du føler deg usikker på hvordan du fullfører
en oppgave.

- Visualiser et flerlagskart ved hjelp av `geopandas.GeoDataFrame.plot()` metoden;
- først, plott befolkningsrutenettet ved hjelp av et 'kvantil' klassifiseringsskjema,
- deretter, legg til veinettverk og tbane-linjer i samme plott (husk `ax`
  parameter)
:::


Husk følgende alternativer som kan sendes til `plot()`:
- stil polygonlaget:
    - definer et klassifiseringsskjema ved hjelp av `scheme` parameteren
    - [endre fargekartet ved hjelp av
      `cmap`](https://matplotlib.org/stable/tutorials/colors/colormaps.html)
    - kontroller lagets gjennomsiktighet med `alpha` parameteren (`0` er fullt
      gjennomsiktig, `1` fullt ugjennomsiktig)
- stil linjelagene:
    - juster [linjefargen](https://matplotlib.org/stable/api/colors_api.html) ved hjelp av
      `color` parameteren
    - endre `linewidth`, etter behov

Lagene har forskjellige omfang (`veier` dekker et større område). Du kan
bruke aksenes (`ax`) metoder `set_xlim()` og `set_ylim()` for å sette den horisontale
og vertikale utstrekningen av kartet (f.eks. til en geodataframes `total_bounds`).

## Legge til en legend/tegnforklaring

For å plotte en legend/tegnforklaring for et kart, legg til parameteren `legend=True`.

For figurer uten et klassifiserings-`scheme`, består legenden av en farget
gradient bar. Legend er en instans av
[`matplotlib.pyplot.colorbar.Colorbar`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html),
og alle argumenter definert i `legend_kwds` blir sendt gjennom til den. Se nedenfor
hvordan du bruker `label` egenskapen for å sette *legend tittelen*:

:::{admonition} Sett andre `Colorbar` parametere
:class: hint

Sjekk ut [`matplotlib.pyplot.colorbar.Colorbar`’s
dokumentasjon](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html)
og eksperimenter med andre parametere! Alt du legger til i `legend_kwds`
ordboken vil bli sendt til fargebaren.
:::


---

For figurer som bruker et klassifisering `scheme`, på den annen side, `plot()`
lager en
[`matplotlib.legend.Legend`](https://matplotlib.org/stable/api/legend_api.html#matplotlib.legend.Legend).
Igjen, `legend_kwds` blir sendt gjennom, men parameterne er litt
forskjellige: for eksempel, bruk `title` i stedet for `label` for å sette legenden
tittel:

:::{admonition} Sett andre `Legend` parametere
:class: hint

Sjekk ut [`matplotlib.pyplot.legend.Legend`’s
dokumentasjon](https://matplotlib.org/stable/api/legend_api.html#matplotlib.legend.Legend),
og eksperimenter med andre parametere! Alt du legger til i `legend_kwds`
ordboken vil bli sendt til legenden.

Hvilket `legend_kwds` nøkkelord vil spre legenden over to kolonner?
:::


## Legge til et basekart

For bedre orientering er det ofte nyttig å legge til et basekart i et kartplott. En
basekart, for eksempel, fra kartleverandører som
[OpenStreetMap](https://osm.org/) eller [Stamen](https://maps.stamen.com/), legger til
gater, stedsnavn, og annen kontekstuell informasjon.

Python-pakken [contextily](https://contextily.readthedocs.io/) tar seg av
nedlasting av nødvendige kartfliser og gjengir dem i en geopandas plot.

:::{admonition} Web Mercator
:class: caution

Kartfliser fra online kartleverandører er typisk i [Web Mercator-projeksjon
(EPSG:3857)](http://spatialreference.org/ref/sr-org/epsg3857-wgs84-web-mercator-auxiliary-sphere/).
Det er generelt tilrådelig å transformere alle andre lag til `EPSG:3857`, også.
:::

For å legge til et bakgrunnskart i et eksisterende plot, bruk
[`contextily.add_basemap()`](https://contextily.readthedocs.io/en/latest/intro_guide.html)
funksjonen, og gi plotets `ax`-objektet som ble lagd i et tidligere trinn.

[Det er mange
andre nettbaserte kart å velge
fra](https://contextily.readthedocs.io/en/latest/intro_guide.html#Providers).
Alle de andre `contextily.providers` (se lenke ovenfor) kan sendes som en
`source` til `add_basemap()`. Du kan få en liste over tilgjengelige leverandører:

På dette zoomnivået, lever fordelene ved å bruke OpenStreetMap (som stedsnavn)
ikke opp til sitt fulle potensial. La oss se på et delsett av reisetidsmatrisen:
befolkningsdataene som er har bare 15 innbyggere.

Til slutt kan vi endre tilskrivningen (copyright-meldingen) som vises i
nederste venstre hjørne av kartplottet. Merk at du alltid skal respektere kart
leverandørenes bruksvilkår, som vanligvis inkluderer en datakilde-attribusjon
(*contextily*’s standarder tar hånd om dette). Vi kan og bør imidlertid
legge til en datakilde for alle lag vi la til, sånn som befolkningsdatasettet: